In [206]:
# !pip install implicit

In [207]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

In [208]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [209]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [210]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [211]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [212]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [213]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [214]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 95.4 ms, sys: 1.99 ms, total: 97.4 ms
Wall time: 100 ms


,user_id,actual,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [215]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items_weights = np.array(items_weights)
    recs = np.random.choice(items_weights, size=n, replace=False)
    
    return recs.tolist()

In [216]:
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights = np.log1p(items_weights)
items_weights = items_weights.rename(columns={'sales_value': 'weight'})
items_weights = items_weights.loc[:, 'weight']

In [217]:
%%time

# items = data_train['item_id'].unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 3.05 s, sys: 13 ms, total: 3.06 s
Wall time: 3.07 s


,user_id,actual,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[2.770711962226921, 1.9110228900548727, 1.2781..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5.178294323948306, 1.095273387402595, 5.02532..."


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [218]:
# result = pd.read_csv('predictions_basic.csv')
# result.head(2)

**(!) Важно** 
- У item-item алгоритмов большая сложность predict ($O(I^2 log(I))$ или $O(I^3)$, в зависимости от реализации 
- Если в датасете много item_id, то item-item модели ОЧЕНЬ долго предсказывают. Со всеми товарами predict на тесте ~2 часа
- Мы возьмем из ~90к товаров только 5k самых популярных 

*P.S.*  Брать топ-Х популярных и рекомендовать только из них - очень популярная стратегия.   
*P.P.S.*  В рекомендательных системах много таких трюков. Что-то подобное в курсе вы увидите еще не раз

In [219]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [220]:
# popularity.sort_values('n_sold', ascending=False)

Возьмём только 5000 самых популярных товаров

In [221]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
print(top_5000[:10])

[6534178, 6533889, 6534166, 6544236, 1404121, 397896, 1426702, 5703832, 480014, 5668996]


In [222]:
# data_train['item_id'].isin(top_5000)

In [223]:
# Заведем фиктивный item_id, если юзер не покупал товары из топ-5000, 
# то присвоим товару id 999999
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [224]:
user_item_matrix.shape

(2499, 5001)

Плотность матрицы

In [227]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

Создадим словари перевода идентификаторов в индексы матрицы и обратно

In [228]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [229]:
# print(id_to_itemid)
# print(id_to_userid)
# print(itemid_to_id)
# print(userid_to_id)

**ItemItemRecommender**

In [230]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.37 s, sys: 8.48 ms, total: 1.37 s
Wall time: 884 ms


In [231]:
recs

(array([3408, 2148, 3947, 2307, 3587], dtype=int32),
 array([56269., 43806., 17219., 12981.,  7431.]))

In [232]:
[id_to_itemid[rec] for rec in recs[0]]

[1082185, 981760, 1127831, 995242, 1098066]

In [233]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 89.8 ms, sys: 0 ns, total: 89.8 ms
Wall time: 92.8 ms


In [234]:
result.head(3)

,user_id,actual,popular_recommendation,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[2.770711962226921, 1.9110228900548727, 1.2781...","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5.178294323948306, 1.095273387402595, 5.02532...","[1082185, 981760, 1127831, 995242, 1098066]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 6533889, 1029743, 6534166, 1082185]","[3.6130777406976544, 2.63834278867739, 1.00795...","[1082185, 981760, 1127831, 995242, 1098066]"


**CosineRecommender**

In [235]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.004546403884887695 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.39 s, sys: 9.37 ms, total: 1.39 s
Wall time: 931 ms


In [236]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 76.8 ms, sys: 0 ns, total: 76.8 ms
Wall time: 79 ms


In [237]:
result.head(3)

,user_id,actual,popular_recommendation,weighted_random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[2.770711962226921, 1.9110228900548727, 1.2781...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5.178294323948306, 1.095273387402595, 5.02532...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 6533889, 1029743, 6534166, 1082185]","[3.6130777406976544, 2.63834278867739, 1.00795...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"


**TFIDFRecommender**

In [238]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.003970623016357422 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.4 s, sys: 7.22 ms, total: 1.4 s
Wall time: 941 ms


In [239]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 74.8 ms, sys: 0 ns, total: 74.8 ms
Wall time: 76.6 ms


In [240]:
result.head(2)

,user_id,actual,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[2.770711962226921, 1.9110228900548727, 1.2781...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5.178294323948306, 1.095273387402595, 5.02532...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"


**BM25Recommender**

In [241]:
%%time

model = BM25Recommender(K=5, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0038678646087646484 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.39 s, sys: 9.63 ms, total: 1.4 s
Wall time: 917 ms


In [242]:
%%time

result['bm25'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 81.4 ms, sys: 0 ns, total: 81.4 ms
Wall time: 86.3 ms


In [243]:
result.head(2)

,user_id,actual,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[2.770711962226921, 1.9110228900548727, 1.2781...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[872137, 901062, 986947, 1098248, 995785, 1082..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5.178294323948306, 1.095273387402595, 5.02532...","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[872137, 901062, 986947, 1098248, 995785, 1082..."


Рассчитаем метрики Precision@5, Recall@5 для каждого алгоритма


In [244]:
def precision_at_k(recommended_list, bought_list, k):
    
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list)
  
  recommended_list = recommended_list[:k]
  
  flags = np.isin(bought_list, recommended_list)
  
  precision = flags.sum() / len(recommended_list)
  
  
  return precision

In [245]:
def recall_at_k(recommended_list, bought_list, k):
    
  bought_list = np.array(bought_list)
  recommended_list = np.array(recommended_list[:k])
  
  flags = np.isin(bought_list, recommended_list)
  
  recall_at_k = flags.sum() / len(bought_list)

  return recall_at_k

Метрики для популярных товаров

In [246]:
precision = result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['popular_recommendation'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 15.52
Recall = 2.50


Для рекомендованных по весам. Эта рекомендация посчитана неправильно!

In [247]:
precision = result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['weighted_random_recommendation'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 0.00
Recall = 0.00


Метрики рекомендации модели itemitem

In [248]:
precision = result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 14.57
Recall = 1.62


Метрики рекомендации модели cosine

In [249]:
precision = result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['cosine'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 13.55
Recall = 1.40


Метрики рекомендации модели tfidf

In [250]:
precision = result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 13.55
Recall = 1.40


Метрики рекомендации модели bm25

In [251]:
precision = result.apply(lambda row: precision_at_k(row['bm25'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['bm25'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 4.41
Recall = 0.40


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Поменяем число соседей в itemitem

In [267]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.33 s, sys: 15.9 ms, total: 1.35 s
Wall time: 837 ms


In [268]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 70.5 ms, sys: 942 µs, total: 71.4 ms
Wall time: 74.6 ms


In [269]:
result.head(1)

,user_id,actual,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[2.770711962226921, 1.9110228900548727, 1.2781...","[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 840361]","[901062, 872137, 986947, 860776, 995785, 1082185]"


In [270]:
precision = result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 16.23
Recall = 1.83


Поменяем число соседей в cosine

In [271]:
%%time

model = CosineRecommender(K=1, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.004289865493774414 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.36 s, sys: 10.6 ms, total: 1.37 s
Wall time: 869 ms


In [272]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 69.5 ms, sys: 1.85 ms, total: 71.4 ms
Wall time: 73.4 ms


In [273]:
precision = result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['cosine'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 13.59
Recall = 1.57


Поменяем число соседей в tfidf

In [277]:
%%time

model = TFIDFRecommender(K=8, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.003888368606567383 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.39 s, sys: 9.82 ms, total: 1.4 s
Wall time: 869 ms


In [278]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 80.2 ms, sys: 0 ns, total: 80.2 ms
Wall time: 85.2 ms


In [279]:
precision = result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 14.67
Recall = 1.55


Поменяем число соседей в bm25

In [283]:
%%time

model = BM25Recommender(K=8, num_threads=4) # K - кол-во ближайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

/usr/local/lib/python3.9/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0040094852447509766 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.42 s, sys: 18.6 ms, total: 1.43 s
Wall time: 897 ms


In [284]:
%%time

result['bm25'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

CPU times: user 97.5 ms, sys: 3.01 ms, total: 101 ms
Wall time: 101 ms


In [285]:
precision = result.apply(lambda row: precision_at_k(row['bm25'], row['actual'], k=5), axis=1).mean() * 100
recall = result.apply(lambda row: recall_at_k(row['bm25'], row['actual'], k=5), axis=1).mean() * 100

print(f'Precision = {precision:.2f}')
print(f'Recall = {recall:.2f}')

Precision = 5.97
Recall = 0.52
